In [1]:
import great_expectations as gx
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os
import json

In [2]:
context = gx.get_context()

spark = SparkSession \
    .builder \
    .appName("klaus_session") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/20 18:12:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/20 18:12:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/20 18:12:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = spark.read.parquet("/opt/airflow/silver_layer/")

In [4]:
dataframe_datasource = context.sources.add_or_update_spark(
    name="spark_memory"
)

In [5]:
data_asset = dataframe_datasource.add_dataframe_asset(name="df_asset", dataframe=df)

In [6]:
my_batch_request = data_asset.build_batch_request()

In [7]:
expectation_name = "data_quality_expectations"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_name)

{
  "data_asset_type": null,
  "expectation_suite_name": "data_quality_expectations",
  "ge_cloud_id": null,
  "meta": {
    "great_expectations_version": "0.18.18"
  },
  "expectations": []
}

In [8]:
validator = context.get_validator(
    batch_request=my_batch_request,
    expectation_suite_name=expectation_name
)

In [9]:
columns_to_check_null = [
    'id_brewery'
]

In [11]:
output_dir = "/opt/airflow/scripts/"
output_file = os.path.join(output_dir, "validation_results.txt")

os.makedirs(output_dir, exist_ok=True)

successful_validations = 0
with open(output_file, 'w') as f:

    validator.expect_column_values_to_not_be_null(column="id_brewery")

    validator.expect_column_values_to_be_unique(column="id_brewery")

    validator.expect_column_value_lengths_to_equal(column="phone_brewery", value=11)

    validator.expect_column_values_to_be_between(column="latitude_brewery", min_value=-90.0, max_value=90.0)

    validator.expect_column_value_lengths_to_be_between(column="postal_code_brewery", min_value=5, max_value=10)

    results = validator.validate()
    total_validations = len(results['results'])

    for result in results['results']:
        expectation_type = result['expectation_config']['expectation_type']
        column = result['expectation_config']['kwargs'].get('column', 'N/A')
        success = result['success']
        
        if success:
            successful_validations += 1
            
        output = f"Expectativa: {expectation_type} | Coluna: {column} | Sucesso: {success}\n"
        f.write(output)  
        
        if not success:
            #failure_details = f"Detalhes da falha: {result['result']}\n"
            failure_details = f"Detalhes da falha: {result}\n"

            f.write(failure_details)
        
        f.write('-' * 60 + '\n')

    success_percentage = (successful_validations / total_validations) * 100

    f.write(f"Percentual de validações bem-sucedidas: {success_percentage:.2f}%\n")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/34 [00:00<?, ?it/s]